In [1]:
from icecream import ic
import torch
import math
import numpy as np

# loss_function   
def loss_function(y_pred, y_target):
     return (y_pred - y_target).pow(2).sum()
    
# (d loss)/(d y_pred) = 2*(y_pred - y_target)
def loss_backwards(y_pred, y_target):
     return 2*(y_pred - y_target)
    
    
dtype = torch.float

In [2]:

# forward_function
def forward_function(x, a, b):
    # y1 = a+b*x1
    # y2 = a+b*x2
    # ....
     return a + b * x
    
# (d forward_function)/(d a) = (1, 1, ...)
def a_backwars(x, a, b):
     return torch.ones(x.shape, dtype = dtype)
     #return torch.tensor([1 for _ in range(len(forward_function(x, a, b)))], dtype = dtype)
    
# (d forward_function)/(d b) = (x1, x2, ...)
def b_backwars(x, a, b):
     return x
    
    
#input
x = torch.tensor([0, 1], dtype = dtype)
#target
y_target = torch.tensor([1, 0], dtype = dtype)

#parameters
a = torch.randn((), dtype=dtype, requires_grad=True)
b = torch.randn((), dtype=dtype, requires_grad=True)

    
ic(forward_function(x, a, b))
y_pred = forward_function(x, a, b)
loss = loss_function(y_pred, y_target)

# Use autograd to compute the backward pass. This call will compute the
# gradient of loss with respect to all Tensors with requires_grad=True.
# After this call a.grad, b.grad. c.grad and d.grad will be Tensors holding
# the gradient of the loss with respect to a, b, c, d respectively.
loss.backward()



ic(" ")
ic("backprop. of a (manually):", (loss_backwards(y_pred, y_target) @ a_backwars(x, a, b)).item())
ic("backprop. of a (with autograd):", a.grad)
ic(" ")
ic("backprop. of a (manually):", (torch.t(a_backwars(x, a, b)) @ loss_backwards(y_pred, y_target)).item())
ic("backprop. of a (with autograd):", a.grad)
ic(" ")
ic("backprop. of b (manually):", (loss_backwards(y_pred, y_target) @ b_backwars(x, a, b)).item())
ic("backprop. of b (with autograd):", b.grad)
ic(" ")
ic("backprop. of b (manually):", (torch.t(b_backwars(x, a, b)) @ loss_backwards(y_pred, y_target)).item())
ic("backprop. of b (with autograd):", b.grad)
ic(b.grad)
ic(" ")




ic| forward_function(x, a, b): tensor([2.3099, 1.9684], grad_fn=<AddBackward0>)
ic| ' '
ic| "backprop. of a (manually):": 'backprop. of a (manually):'
    (loss_backwards(y_pred, y_target)@a_backwars(x, a, b)).item(): 6.5564422607421875
ic| 'backprop. of a (with autograd):', a.grad: tensor(6.5564)
ic| ' '
ic| "backprop. of a (manually):": 'backprop. of a (manually):'
    (torch.t(a_backwars(x, a, b))@loss_backwards(y_pred, y_target)).item(): 6.5564422607421875
ic| 'backprop. of a (with autograd):', a.grad: tensor(6.5564)
ic| ' '
ic| "backprop. of b (manually):": 'backprop. of b (manually):'
    (loss_backwards(y_pred, y_target)@b_backwars(x, a, b)).item(): 3.936736822128296
ic| 'backprop. of b (with autograd):', b.grad: tensor(3.9367)
ic| ' '
ic| "backprop. of b (manually):": 'backprop. of b (manually):'
    (torch.t(b_backwars(x, a, b))@loss_backwards(y_pred, y_target)).item(): 3.936736822128296
ic| 'backprop. of b (with autograd):', b.grad: tensor(3.9367)
ic| b.grad: tensor(3.9367)
i

' '

In [3]:

# forward_function
def forward_function(x, w, b):
    # y1 = b1+w(1,1)*x1 + w(1,2)*x2
    # y2 = b2+w(2,1)*x1 + w(2,2)*x2
     return w @ x + b
    
# (d forward_function)/(d w) = (1, 1, ...)
#def w_backwars(x, w, b):
#     return torch.ones(len(x), dtype = dtype)
    
# (d forward_function)/(d b) = (x1, x2, ...)
def b_backwars(x, w, b):
    #(d y)/(d b) = ((d y1)/(d b), (d y2)/(d b)) 
    # = (((d y1)/(d b1), (d y1)/(d b2)), ((d y2)/(d b1), (d y2)/(d b2))) #Jacobimatrix
    # = ((1, 0), (0, 1))
    return torch.eye(len(b), dtype = dtype)

def w_backwars(x, w, b):
    # (d y)/(d w) = ((d y1)/(d w), (d y2)/(d w)) 
    # = ((((d y1)/(d w(1,1)), (d y1)/(d w(1,2))), ((d y1)/(d w(2,1)), (d y1)/(d w(2,2)))), (((d y2)/(d w(1,1)), (d y2)/(d w(1,2))), ((d y2)/(d w(2,1)), (d y2)/(d w(2,2))))) #Von uns veralgemeinerte Jacobimatrix (Tensor 3.stufe)
    # = ((( x1              ,  x2              ), ( 0               ,  0               )), (( 0               ,  0               ), ( x1              ,  x2              )))
    # = ([(x1, x2), (0,  0)], [(0,  0), (x1,  x2)])
    # 
    c = torch.zeros(len(x), len(x), len(x), dtype=x.dtype) 
    for i in range(len(x)):
        c[i,i] = x
    return c
    #return x.repeat(len(x),1) # does not work
    
    
#input
#x = torch.tensor([0, 1], dtype = dtype)
x = torch.randn((2), dtype=dtype, requires_grad=True)

#target
#y_target = torch.tensor([1, 0], dtype = dtype)
y = torch.randn((2), dtype=dtype, requires_grad=True)

#parameters
#w = torch.arange(4).reshape(2,2)
#w = torch.tensor(w.clone(), dtype=dtype, requires_grad=True)
#b = torch.arange(2, dtype=dtype, requires_grad=True)

w = torch.randn((2,2), dtype=dtype, requires_grad=True)
b = torch.randn((2), dtype=dtype, requires_grad=True)
    
y_pred = forward_function(x, w, b)
loss = loss_function(y_pred, y_target)

# Use autograd to compute the backward pass. This call will compute the
# gradient of loss with respect to all Tensors with requires_grad=True.
# After this call a.grad, b.grad. c.grad and d.grad will be Tensors holding
# the gradient of the loss with respect to a, b, c, d respectively.
loss.backward()

ic(loss_backwards(y_pred, y_target),w_backwars(x, w, b) )
test = w_backwars(x, w, b)[0] * loss_backwards(y_pred, y_target)[0] + w_backwars(x, w, b)[1] * loss_backwards(y_pred, y_target)[1]
ic(test)
ic(" ")
ic("backprop. of w (manually):", (loss_backwards(y_pred, y_target) @ w_backwars(x, w, b)))
ic("backprop. of w (with autograd):", w.grad)
ic(" ")
ic("backprop. of b (manually):", (loss_backwards(y_pred, y_target) @ b_backwars(x, w, b)))
ic("backprop. of b (with autograd):", b.grad)
#ic(b.grad)
ic(" ")

ic| loss_backwards(y_pred, y_target): tensor([-1.6441,  2.2064], grad_fn=<MulBackward0>)
    w_backwars(x, w, b): tensor([[[-0.1499, -0.3289],
                                  [ 0.0000,  0.0000]],
                         
                                 [[ 0.0000,  0.0000],
                                  [-0.1499, -0.3289]]], grad_fn=<CopySlices>)
ic| test: tensor([[ 0.2464,  0.5407],
                  [-0.3306, -0.7256]], grad_fn=<AddBackward0>)
ic| ' '
ic| "backprop. of w (manually):": 'backprop. of w (manually):'
    loss_backwards(y_pred, y_target)@w_backwars(x, w, b): tensor([[ 0.2464,  0.5407],
                                                                  [-0.3306, -0.7256]], grad_fn=<UnsafeViewBackward0>)
ic| "backprop. of w (with autograd):": 'backprop. of w (with autograd):'
    w.grad: tensor([[ 0.2464,  0.5407],
                    [-0.3306, -0.7256]])
ic| ' '
ic| "backprop. of b (manually):": 'backprop. of b (manually):'
    loss_backwards(y_pred, y_target)@b_back

' '

In [4]:
# forward_function
def forward_function(x, a, b):
    # y1 = a+b*x1
    # y2 = a+b*x2
    # ....
     return a + b * x
    
# (d forward_function)/(d a) = (1, 1, ...)
def a_backwars(x, a, b):
     return np.ones(x_np.shape)
     #return torch.tensor([1 for _ in range(len(forward_function(x, a, b)))], dtype = dtype)
    
# (d forward_function)/(d b) = (x1, x2, ...)
def b_backwars(x, a, b):
     return x
    
#input
x_np = np.random.rand(2)
x = torch.tensor(x_np, dtype = dtype)

#target
y_target_np = np.random.rand(2)
y_target = torch.tensor(y_target_np, dtype = dtype)

#parameters
a_np = np.random.rand()
a = torch.tensor(a_np, dtype = dtype, requires_grad=True)
b_np = np.random.rand()
b = torch.tensor(b_np, dtype = dtype, requires_grad=True)
    
ic(forward_function(x, a, b))

y_pred_np = forward_function(x_np, a_np, b_np)
y_pred = forward_function(x, a, b)
loss = loss_function(y_pred, y_target)

# Use autograd to compute the backward pass. This call will compute the
# gradient of loss with respect to all Tensors with requires_grad=True.
# After this call a.grad, b.grad. c.grad and d.grad will be Tensors holding
# the gradient of the loss with respect to a, b, c, d respectively.
loss.backward()




ic(" ")
ic("backprop. of a (manually):", (loss_backwards(y_pred_np, y_target_np) @ a_backwars(x_np, a_np, b_np)).item())
ic("backprop. of a (with autograd):", a.grad)
ic(" ")
ic("backprop. of a (manually):", (loss_backwards(y_pred_np, y_target_np) @ b_backwars(x_np, a_np, b_np)).item())
ic("backprop. of b (with autograd):", b.grad)
ic(" ")




ic| forward_function(x, a, b): tensor([0.7106, 0.2854], grad_fn=<AddBackward0>)
ic| ' '
ic| "backprop. of a (manually):": 'backprop. of a (manually):'
    (loss_backwards(y_pred_np, y_target_np)@a_backwars(x_np, a_np, b_np)).item(): -0.2600867767409524
ic| 'backprop. of a (with autograd):', a.grad: tensor(-0.2601)
ic| ' '
ic| "backprop. of a (manually):": 'backprop. of a (manually):'
    (loss_backwards(y_pred_np, y_target_np)@b_backwars(x_np, a_np, b_np)).item(): 0.1485618433179697
ic| 'backprop. of b (with autograd):', b.grad: tensor(0.1486)
ic| ' '


' '

In [4]:
# forward_function
def forward_function(x, w, b):
    # y1 = b1+w(1,1)*x1 + w(1,2)*x2
    # y2 = b2+w(2,1)*x1 + w(2,2)*x2
     return w @ x + b
    
# (d forward_function)/(d w) = (1, 1, ...)
#def w_backwars(x, w, b):
#     return torch.ones(len(x), dtype = dtype)
    
# (d forward_function)/(d b) = (x1, x2, ...)
def b_backwars(x, w, b):
    #(d y)/(d b) = ((d y1)/(d b), (d y2)/(d b)) 
    # = (((d y1)/(d b1), (d y1)/(d b2)), ((d y2)/(d b1), (d y2)/(d b2))) #Jacobimatrix
    # = ((1, 0), (0, 1))
    return np.eye(len(b)) # note len(y) = len(b) = # rows of w, because y = w*x +b

def w_backwars(x, w, b):
    # calculation for dimensions: w - 2x2, b - 2, x -2
    # (d y)/(d w) = ((d y1)/(d w), (d y2)/(d w)) 
    # = ((((d y1)/(d w(1,1)), (d y1)/(d w(1,2))), ((d y1)/(d w(2,1)), (d y1)/(d w(2,2)))), (((d y2)/(d w(1,1)), (d y2)/(d w(1,2))), ((d y2)/(d w(2,1)), (d y2)/(d w(2,2))))) #Von uns veralgemeinerte Jacobimatrix (Tensor 3.stufe)
    # = ((( x1              ,  x2              ), ( 0               ,  0               )), (( 0               ,  0               ), ( x1              ,  x2              )))
    # = ([(x1, x2), (0,  0)], [(0,  0), (x1,  x2)])
    # formaly: we have a function: R⁴ -> R², our Jacobian would be 4 x 2 matrix (the most inner dimension would
    # be flatten out): J = ([(x1, x2, 0,  0)], [(0,  0, x1,  x2)]), with this the standard mathmatics works and the matrix products are defined
    # but to keep dimensions of the matrix, we have the extra dimensions and the multiplication works, because the implentation 
    # of the matrix multiplication in pytorch and numpy cover the case of 1d-matrix @ 3d-matrix, and it does what we want, and it saves us from reshaping later 
    c = np.zeros((len(w), len(w), len(x)))
    #= ((((d y1)/(d w(1,1)), (d y1)/(d w(1,2))), ((d y1)/(d w(2,1)), (d y1)/(d w(2,2)))), (((d y2)/(d w(1,1)), (d y2)/(d w(1,2))), ((d y2)/(d w(2,1)), (d y2)/(d w(2,2)))))
    #= ([(x1, x2), (0,  0), (0, 0)], [(0,  0), (x1,  x2), (0, 0)], [(0,  0), (0,  0), (x1, x2)] )
    for i in range(len(w)):
        c[i,i] = x
    return c
    #return x.repeat(len(x),1) # does not work
    
def x_backwards(x, w, b):
    #(d y)/(d x) = ((d y1)/(d x), (d y2)/(d x)) 
    # = (((d y1)/(d x1), (d y1)/(d x2)), ((d y2)/(d x1), (d y2)/(d x2)))
    # = (( w(1,1)      ,  w(1,2)      ), ( w(2,1),        w(2,2)      )) = w
    return w
    

    

#input
x_np = np.random.rand(2)
x = torch.tensor(x_np, dtype = dtype, requires_grad=True)

#target
y_target_np = np.random.rand(3)
y_target = torch.tensor(y_target_np, dtype = dtype)

#parameters
w_np = np.random.rand(3,2)
w = torch.tensor(w_np, dtype=dtype, requires_grad=True)
b_np = np.random.rand(3)
b = torch.tensor(b_np, dtype = dtype, requires_grad=True)

    
y_pred = forward_function(x, w, b)
y_pred_np = forward_function(x_np, w_np, b_np)
loss = loss_function(y_pred, y_target)

# Use autograd to compute the backward pass. This call will compute the
# gradient of loss with respect to all Tensors with requires_grad=True.
# After this call a.grad, b.grad. c.grad and d.grad will be Tensors holding
# the gradient of the loss with respect to a, b, c, d respectively.

loss.backward()

ic(" ")
ic("backprop. of w (manually):", (loss_backwards(y_pred_np, y_target_np) @ w_backwars(x_np, w_np, b_np)))
ic("backprop. of w (with autograd):", w.grad)
ic(" ")  
ic("backprop. of b (manually):", (loss_backwards(y_pred_np, y_target_np) @ b_backwars(x_np, w_np, b_np)))
ic("backprop. of b (with autograd):", b.grad)
ic(" ")
ic("backprop. of b (manually):", (loss_backwards(y_pred_np, y_target_np) @ x_backwards(x_np, w_np, b_np)))
ic("backprop. of b (with autograd):", x.grad)
ic(" ")


ic| ' '
ic| "backprop. of w (manually):": 'backprop. of w (manually):'
    loss_backwards(y_pred_np, y_target_np) @ w_backwars(x_np, w_np, b_np): array([[1.1763296 , 0.55752889],
                                                                                  [1.36888214, 0.64879039],
                                                                                  [0.36036128, 0.17079552]])
ic| "backprop. of w (with autograd):": 'backprop. of w (with autograd):'
    w.grad: tensor([[1.1763, 0.5575],
                    [1.3689, 0.6488],
                    [0.3604, 0.1708]])
ic| ' '
ic| "backprop. of b (manually):": 'backprop. of b (manually):'
    loss_backwards(y_pred_np, y_target_np) @ b_backwars(x_np, w_np, b_np): array([1.2963876 , 1.50859235, 0.39714031])
ic| "backprop. of b (with autograd):": 'backprop. of b (with autograd):'
    b.grad: tensor([1.2964, 1.5086, 0.3971])
ic| ' '
ic| "backprop. of b (manually):": 'backprop. of b (manually):'
    loss_backwards(y_pred_np, y_targe

' '

In [82]:
# scratch:
c = np.zeros((3, 2))
x = np.array([1,2])
print(c,x)
print(c@x)
print(c.size)
print(len(c))
print(np.add(x,x))

print(np.array((2))*np.array((3,3)))
np.array((2,3))@np.array((3,3))

ic(np.array((2)).ndim)
ic(np.float64(2).ndim)
z = ic(np.array([x]))

ic(np.diag(np.diag(z)))
c = np.zeros((len(w), len(w), len(x)))
np.fill_diagonal(c[:,:],x)

ic| np.array((2)).ndim: 0
ic| np.float64(2).ndim: 0
ic| np.array([x]): array([[1, 2]])
ic| np.diag(np.diag(z)): array([[1]])


[[0. 0.]
 [0. 0.]
 [0. 0.]] [1 2]
[0. 0. 0.]
6
3
[2 4]
[6 6]


ValueError: All dimensions of input must be of equal length

[[0. 0.]
 [0. 0.]
 [0. 0.]] [1 2]
[0. 0. 0.]
6
3
[2 4]
